In [ ]:
# @title Подключение к диску с данными
import os
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# @title Добавление новых фич (версия с разделением на 3 части улучшенный mcc target encoder и для 2 других столбцов)
import polars as pl
import gc
pl.Config.set_streaming_chunk_size(100_000)  # Ограничение чанков для streaming

# Пути
DATA_PATH = "/content/drive/MyDrive/ml-vtb-data-fusion-strazh/data/"
# TEMP_PATH = "/content/temp_te/"

PARTS = [1,2,3]

print("Вычисляем глобальное среднее target по всем частям (для сглаживания)")
global_sum = 0
global_cnt = 0
for i in PARTS:
    stats = pl.scan_parquet(f"{DATA_PATH}train_full_part_{i}.parquet") \
              .select(pl.sum("target"), pl.len()) \
              .collect()
    global_sum += stats[0, 0]
    global_cnt += stats[0, 1]
global_target_mean = global_sum / global_cnt
C = 15

for i in PARTS:
    # Загружаем lazyFrame
    df = pl.scan_parquet(f"{DATA_PATH}train_full_part_{i}.parquet")


    # 1. Базовые time-features
    df = df.with_columns([
        pl.col("event_dttm").dt.hour().alias("hour"),
        pl.col("event_dttm").dt.weekday().alias("dow"),
        pl.col("event_dttm").dt.date().alias("date"),
        (pl.col("operaton_amt").log1p()).alias("log_amt")
        ])


    # 2. Device risk score (с fill_null для null)
    df = df.with_columns([
        pl.col("compromised").cast(pl.Int32),
        pl.col("developer_tools").cast(pl.Int32),
        pl.col("web_rdp_connection").cast(pl.Int32),
        pl.col("phone_voip_call_state").cast(pl.Int32)
        ])

    df = df.with_columns(
        pl.sum_horizontal([
            "compromised",
            "developer_tools",
            "web_rdp_connection",
            "phone_voip_call_state"
            ]).alias("device_risk")
            )


    # 3. Rolling features по customer_id (отдельно agg + join)
    windows = ["1h", "24h", "7d"]

    for w in windows:
        # Сортируем (для rolling) так как перед каждой такой операцией нам нужна строгая сортировка
        df = df.sort(["customer_id", "event_dttm"])
        agg = df.rolling(
            index_column="event_dttm",    # временная колонка
            period=w,                      # размер окна
            group_by="customer_id",        # группировка
            closed="left",                 # не включаем текущую строку (только прошлое)
            ).agg([
                pl.col("event_id").count().alias(f"cnt_{w}"),
                pl.col("operaton_amt").sum().alias(f"sum_amt_{w}"),
                pl.col("operaton_amt").mean().alias(f"mean_amt_{w}"),
                pl.col("mcc_code").n_unique().alias(f"uniq_mcc_{w}"),
                ])

        # Join обратно к df
        df = df.join(agg, on=["customer_id", "event_dttm"], how="left")

    # Сохраняем intermediate после rolling (чтобы освободить память перед TE)
    print("Сохраняем intermediate после rolling...")
    df.collect(engine="streaming").write_parquet(f"/content/train_after_rolling_part_{i}.parquet", compression="zstd")
    del df
    gc.collect()

    # Перезагружаем lazy для TE
    df = pl.scan_parquet(f"/content/train_after_rolling_part_{i}.parquet")
    cat_cols = ["mcc_code", "event_type_nm", "channel_indicator_type"]
    for col in cat_cols:
        print(f"TE для {col} (expanding smoothing)...")

        # Сортируем для корректного кумулятивного суммирования
        df = df.sort(["customer_id", col, "event_dttm"])

        # Вычисляем кумулятивные суммы target и номер строки в группе
        df = df.with_columns([
            pl.col("target").cum_sum().over(["customer_id", col]).alias("cum_target_all"),
            pl.int_range(0, pl.len()).over(["customer_id", col]).alias("cum_cnt_all")  # 0,1,2,... в группе
        ])

        # Получаем сумму и количество ТОЛЬКО для предыдущих строк (исключая текущую)
        df = df.with_columns([
            (pl.col("cum_target_all") - pl.col("target")).alias("cum_target_prev"),
            pl.col("cum_cnt_all").alias("cum_cnt_prev")  # для первой строки = 0
        ])

        # Применяем сглаживание с глобальным средним
        df = df.with_columns(
            ((pl.col("cum_target_prev").fill_null(0) + C * global_target_mean) /
            (pl.col("cum_cnt_prev").fill_null(0) + C)).alias(f"te_{col}_30d")
        )

        # Удаляем временные колонки
        df = df.drop(["cum_target_all", "cum_cnt_all", "cum_target_prev", "cum_cnt_prev"])

        # Сохраняем промежуточный результат
        temp_file = f"/content/temp_te_{col}_part_{i}.parquet"
        df.collect(engine="streaming").write_parquet(temp_file, compression="zstd", row_group_size=1000000)
        print(f"Temp сохранён для {col}: {temp_file}")

        # Освобождаем память и перезагружаем для следующей колонки
        del df
        gc.collect()
        df = pl.scan_parquet(temp_file)


    # Финальный save
    print("Финальное сохранение...")
    df.sink_parquet(f"{DATA_PATH}train_features_part_{i}.parquet", compression="zstd", row_group_size=1000000)
    print("Все сохранено  !!!!!!!")

    del df
    gc.collect()